In [9]:
from scapy.all import *

## Filter out RST Packets

In [3]:
!iptables --help

iptables v1.8.7

Usage: iptables -[ACD] chain rule-specification [options]
	iptables -I chain [rulenum] rule-specification [options]
	iptables -R chain rulenum rule-specification [options]
	iptables -D chain rulenum [options]
	iptables -[LS] [chain [rulenum]] [options]
	iptables -[FZ] [chain] [options]
	iptables -[NX] chain
	iptables -E old-chain-name new-chain-name
	iptables -P chain target [options]
	iptables -h (print this help information)

Commands:
Either long or short options are allowed.
  --append  -A chain		Append to chain
  --check   -C chain		Check for the existence of a rule
  --delete  -D chain		Delete matching rule from chain
  --delete  -D chain rulenum
				Delete rule rulenum (1 = first) from chain
  --insert  -I chain [rulenum]
				Insert in chain as rulenum (default 1=first)
  --replace -R chain rulenum
				Replace rule rulenum (1 = first) in chain
  --list    -L [chain [rulenum]]
				List the rules in a chain or all chains
  --list-rules -S [chain [rulenum]]
				Pri

##### Outbound 
```bash
$ iptables -A OUTPUT -p tcp --tcp-flags RST RST -j DROP
```


In [4]:
!ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: wlp0s20f3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000
    link/ether dc:1b:a1:9c:e7:94 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.3/24 brd 192.168.1.255 scope global dynamic noprefixroute wlp0s20f3
       valid_lft 84888sec preferred_lft 84888sec
    inet6 fe80::88fd:98e7:a365:c0f1/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever
3: gpd0: <POINTOPOINT,MULTICAST,NOARP> mtu 1500 qdisc noop state DOWN group default qlen 500
    link/none 


#### TCP RST Attack

In [104]:
ip_host= "192.168.1.3"

In [86]:

# SYN
ip=IP(dst=ip_host)
SYN=TCP(sport=22,dport=22,flags='S',seq=1000)
SYNACK=sr1(ip/SYN,timeout=5)



Begin emission:
Finished sending 1 packets.
*
Received 1 packets, got 1 answers, remaining 0 packets


In [103]:
SYNACK

<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=63 proto=tcp chksum=0x715e src=192.168.1.3 dst=172.17.92.177 |<TCP  sport=ssh dport=ssh seq=1966037757 ack=1001 dataofs=6 reserved=0 flags=SA window=64240 chksum=0xfe75 urgptr=0 options=[('MSS', 1460)] |>>

In [87]:
SYNACK.ack

1001

In [88]:
SYNACK.seq

1966037757

In [89]:
# SYN-ACK
ACK=TCP(sport=sport, dport=22, flags='A', seq=SYNACK.ack + 1, ack=SYNACK.seq + 1)
send(ip/ACK)


Sent 1 packets..


#### Host respond
![alt text](tcp_rst_ack.png "tcp_rst_ack")

#### Now block TCP RST 

```bash
#Inbound

$ iptables -I INPUT -p tcp --tcp-flags ALL RST,ACK -j DROP
$ iptables -I INPUT -p tcp --tcp-flags ALL RST -j DROP

#Outbound

$ iptables -I OUTPUT -p tcp --tcp-flags ALL RST,ACK -j DROP
```

#### Current Rules:
```bash
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:38]
└─[$] <> sudo iptables -L OUTPUT --line-numbers
Chain OUTPUT (policy ACCEPT)
num  target     prot opt source               destination
1    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST,ACK
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:38]
└─[$] <> sudo iptables -L INPUT --line-numbers
Chain INPUT (policy ACCEPT)
num  target     prot opt source               destination
1    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST
2    DROP       tcp  --  anywhere             anywhere             tcp flags:FIN,SYN,RST,PSH,ACK,URG/RST,ACK
(base) ┌─[uninorte@uninorte-dell] - [/code/nets] - [jue jul 28, 15:39]
```

In [98]:
# SYN
ip2=IP(dst=ip_host)
SYN2=TCP(sport=1500,dport=5000,flags='S',seq=1000)
SYNACK2=sr1(ip2/SYN2,timeout=5)


Begin emission:
Finished sending 1 packets.

Received 0 packets, got 0 answers, remaining 1 packets


In [102]:
print(SYNACK2)

None


## TraceRoute

In [11]:
## TCP traceroute

target = ["192.168.1.1"]
result, unans = traceroute(target,maxttl=8)

Begin emission:
Finished sending 8 packets.

Received 0 packets, got 0 answers, remaining 8 packets
 


In [33]:
target = ["dartmouth.edu"]
result, unans = traceroute(target,maxttl=10)

Begin emission:
Finished sending 10 packets.
*******
Received 7 packets, got 7 answers, remaining 3 packets
   129.170.171.44:tcp80 
1  192.168.1.1     11   
2  186.102.160.2   11   
4  190.98.141.28   11   
5  94.142.118.231  11   
6  81.173.107.205  11   
9  4.53.54.50      11   
10 129.170.1.49    11   


In [44]:

def traceroute_tcp(target, maxttl=8):
    for d in range(1, maxttl):
        x = IP(version=4, dst=target, ttl=d)/ICMP()
        y = sr1(x, verbose=0)
        if y.getlayer(ICMP).type == 11 and y.getlayer(ICMP).code == 0:    # time to leave exceeded (11)
            print(f"Iteration {d}: ", y.src)
        elif y.getlayer(ICMP).type == 0 and y.getlayer(ICMP).code == 0:   # echo-reply (0)
            print(f"Iteration {d}: ", y.src," Done!")
            break
        else:
            print(f"Iteration {d}: ", "* * *")
           


In [45]:
traceroute_tcp("192.168.1.1")

Iteration 1:  192.168.1.1  Done!


In [46]:
traceroute_tcp("8.8.8.8")

Iteration 1:  192.168.1.1
Iteration 2:  186.102.160.2
Iteration 3:  72.14.222.115
Iteration 4:  72.14.222.114
Iteration 5:  108.170.253.209
Iteration 6:  142.250.231.163
Iteration 7:  8.8.8.8  Done!


In [28]:
#UDP
def traceroute_udp(host,maxttl=4,dport=33434): 
    for i in range(1, maxttl):
        pkt = IP(dst=host, ttl=i) / UDP(dport=dport)
        # Send the packet and get a reply
        reply = sr1(pkt, verbose=0,timeout=2)
        if reply is None:
            break
        elif reply.type == 3:
            # We've reached our destination
            print (f"{i} Iteration", reply.src," Done!")
            break
        else:
            # We're in the middle somewhere
            print (f"{i} Iteration: ", reply.src)

In [29]:
traceroute_udp("192.168.1.1")

1 Iteration 192.168.1.1  Done!


In [30]:
traceroute_udp("8.8.8.8",maxttl=10)

1 Iteration:  192.168.1.1
2 Iteration:  186.102.160.2
3 Iteration:  72.14.222.115
4 Iteration:  72.14.222.114
5 Iteration:  108.170.253.209
6 Iteration:  142.250.231.163
7 Iteration 8.8.8.8  Done!


In [15]:
def traceroute_icmp(host):
    print("Tracroute", host)
    flag = True
    ttl=1
    hops = []
    while flag:
        ans, unans = sr(IP(dst=host,ttl=ttl)/ICMP(),verbose=0)
        if ans.res[0][1].type == 0: # checking for  ICMP echo-reply
                hops.append(ans.res[0][1].src)
                flag = False
        else:
                hops.append(ans.res[0][1].src) # storing the src ip from ICMP error message
                ttl +=1

    for i,hop in enumerate(hops):
        print (i, " " + hop)
    return hops

In [16]:
_ = traceroute_icmp("8.8.8.8")

Tracroute 8.8.8.8
0  192.168.1.1
1  186.102.160.2
2  72.14.222.115
3  72.14.222.114
4  108.170.253.209
5  142.250.231.163
6  8.8.8.8


In [257]:
res, unans = traceroute(["www.microsoft.com",],dport=[80,443],maxttl=20,retry=-2)

Begin emission:
Finished sending 40 packets.
************************************Begin emission:
Finished sending 4 packets.
Begin emission:
Finished sending 4 packets.

Received 36 packets, got 36 answers, remaining 4 packets
   23.14.36.240:tcp443 23.14.36.240:tcp80 
1  172.17.80.1     11  172.17.80.1     11 
2  192.168.1.1     11  192.168.1.1     11 
3  186.102.160.2   11  186.102.160.2   11 
6  94.142.99.158   11  94.142.99.93    11 
7  23.14.36.240    SA  23.14.36.240    SA 
8  23.14.36.240    SA  23.14.36.240    SA 
9  23.14.36.240    SA  23.14.36.240    SA 
10 23.14.36.240    SA  23.14.36.240    SA 
11 23.14.36.240    SA  23.14.36.240    SA 
12 23.14.36.240    SA  23.14.36.240    SA 
13 23.14.36.240    SA  23.14.36.240    SA 
14 23.14.36.240    SA  23.14.36.240    SA 
15 23.14.36.240    SA  23.14.36.240    SA 
16 23.14.36.240    SA  23.14.36.240    SA 
17 23.14.36.240    SA  23.14.36.240    SA 
18 23.14.36.240    SA  23.14.36.240    SA 
19 23.14.36.240    SA  23.14.36.240    SA 

## MITM Packet

In [24]:
import os
os.system("iptables -I INPUT -d 192.168.1.0/24 -j NFQUEUE --queue-num 1")

0

In [25]:
from scapy.all import *
from netfilterqueue import NetfilterQueue
def print_and_accept(pkt):
    packet = IP(pkt.get_payload())
    
    if UDP in packet:
        packet_type = "UDP"

    if TCP in packet:
        packet_type = "TCP"

    if IP in packet:
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        src_port = packet[IP].sport
        dst_port = packet[IP].dport
    
    packet_len = pkt.get_payload_len()
    log_str = "(" + packet_type + ")" + " " + ip_src + ":" + str(src_port) + " -> " + ip_dst + ":" + str(
            dst_port) + " size = " + str(packet_len)
    print(log_str)

    #print(pkt)
    pkt.accept()
    
nfqueue = NetfilterQueue()
nfqueue.bind(1, print_and_accept)
try:
    nfqueue.run()
except KeyboardInterrupt: #stop using jupyter interrupt kernel signal
    nfqueue.unbind()


(TCP) 34.89.141.94:443 -> 192.168.1.3:42514 size = 52
(TCP) 34.89.141.94:443 -> 192.168.1.3:42514 size = 52
(TCP) 142.250.78.133:443 -> 192.168.1.3:48430 size = 52
(TCP) 35.174.210.7:443 -> 192.168.1.3:38110 size = 100
(TCP) 35.174.210.7:443 -> 192.168.1.3:38110 size = 99
(TCP) 173.194.219.189:443 -> 192.168.1.3:33556 size = 103
(TCP) 173.194.219.189:443 -> 192.168.1.3:33556 size = 52
(TCP) 172.217.173.35:443 -> 192.168.1.3:49976 size = 52
(TCP) 35.174.210.7:443 -> 192.168.1.3:38110 size = 52
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 1278
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 1278
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 656
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 54
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 120
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 57
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 53
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 53
(UDP) 172.217.28.99:443 -> 192.168.1.3:53172 size = 997


In [26]:
os.system("iptables -F")
os.system("iptables -F -t nat")

0